In [ ]:
import msmrd2
import numpy as np

In [ ]:
# Define continuous-time MSM
MSMtype = 1
ratematrix = np.array([[-5.0,2.0,3.0],[1.0,-2.0,1.0],[3.3,5.4,-8.7]])
seed = 0 # Seed = -1 used random device as seed
ctmsm = msmrd2.ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([1.0, 3.0, 2.0])
Drotlist = np.array([0.5, 0.2, 0.1])
ctmsm.setD(Dlist)
ctmsm.setDrot(Drotlist)

In [ ]:
# Particle definition
ptype = 0
pstate = 0
bodytype = 'rigidbody'
D = Dlist[pstate]
Drot = Drotlist[pstate]
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particleMS(ptype, pstate, D, Drot, bodytype, position, orientation)

In [ ]:
# Over-damped Langevin integrator with Markovian switch definition
dt = 0.05 # should be smaller than Gillespie inverse transition rates
seed = 0 # Seed = -1 used random device as seed
rotation = True
intg = msmrd2.integrators.odLangevinMarkovSwitch(ctmsm, dt, seed, rotation) 
# Create second integrator for list integration
intg2 = msmrd2.integrators.odLangevinMarkovSwitch(ctmsm, dt, seed, rotation) 

In [ ]:
# Integrate only one particle and print position and orientation
timesteps = 50
print('{:<10s}{:<15s}{:<15s}{:<15s}{:<15s}{:<40s}'.format("Iteration", "Time", "State", "D", "Drot", "Position 1"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<15d}{:<15f}{:<15f}{:<40s}'
          .format(i, intg.clock, part1.state, part1.D, part1.Drot, str(part1.position)))
    intg.integrate(part1)

In [ ]:
# For a list of particles
position2 = np.array([2,2,2])
part1 = msmrd2.particleMS(ptype, pstate, D, Drot, bodytype, position, orientation)
part2 = msmrd2.particleMS(ptype, pstate, D, Drot, bodytype, position2, orientation)
partlist = msmrd2.integrators.particleMSList([part1,part2])
# Integrate particle list and print only positions (with integratelist)
t = 0
timesteps = 50
print('{:<10s}{:^10s}{:<40s}{:^15s}{:<30s}'.format("Time", "State 1", "Position 1", "State 2", "Position2"))
for i in range(timesteps):
    print('{:<10f}{:^10d}{:<40s}{:^15d}{:<30s}'
          .format(intg2.clock, partlist[0].state, str(partlist[0].position), partlist[1].state, str(partlist[1].position)))
    intg2.integrateList(partlist)